# Assignment Algorithms : Part 2

**WARNING**: this notebook takes considerably more time and memory than most tutorials here,
and is probably best studied during a time other than a DESI collaboration
meeting tutorial session.

In the first part of this study we looked at simulated targets with realistic uniform densities and no physical clustering.  We gave those simulated targets realistic priorities and requested observations, and used a realistic focalplane model with accurate positioner motions and exclusion zones.  We also studied just a single pointing on the sky with one or more coincident tiles and this allowed us to Monte Carlo over the simulated targets to see the spread of the assignment fractions.

This second part of the study starts with a cutout of DR8 data and uses that for a study of fiber assignment with multiple passes.  Some things to keep in mind when working with real targets:

1.  Clustering effects will reduce assignment efficiency compared to the study in Part 1, due to an increase in positioner collisions.

2.  Recall that when computing the "total priority" of targets with the same `PRIORITY` value, targets with more remaining observations will be attempted first (see https://github.com/desihub/fiberassign/issues/196).  This means that "4 pass QSOs" will have their assignment attempted first until their remaining requested observations equals "1", at which point their total priority relative to the other "1 pass QSOs" will be based on the random `SUBPRIORITY` value.  Obviously all QSOs will be attempted before the other lower priority target classes.

3.  Real targets include some complications like targets that are marked as both science targets and standards.  These targets are assigned as science targets, but that assignment counts towards the standards budget.  Even after such a target has zero remaining observations, it can still be assigned as a standard.

*This notebook was motivated by a study done by Claire Lamman.  It has been expanded to illustrate more of the details.*

## Fiber Assignment vs. Survey Planning

Fiberassign by itself is a toolkit for taking an input set of tiles and an input set of MTL catalogs and producing the "best possible" assignment of those tiles given the physical hardware and the pre-determined priorities and requested observations of the input targets.

The overall assignment efficiency for the entire survey depends greatly on how exactly you choose to run fiberassign on different sets of tiles and how you update the target files in between those runs.  For the real survey, we will have a feedback loop at some cadence going like this:
```
Targeting ---> make_mtl() ---> MTL ---> Fiberassign ---> Observations ---> Spectro Pipeline
                                ^                                                 |
                                |                                                 V
                                +--<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<-<--+
```
Any time an "assignment efficiency" is quoted for the survey, it must also include information about how this loop was operated.


## Imports and Definitions

These are global imports and variables used throughout the notebook.

In [ ]:
import os
import sys
from collections import OrderedDict
import shutil
from datetime import datetime
import glob
import re

import numpy as np
from numpy.lib.recfunctions import append_fields

import matplotlib.pyplot as plt
%matplotlib inline

from scipy.spatial import KDTree

import fitsio

from desimodel.io import findfile as dm_findfile
from desimodel.io import load_tiles as dm_load_tiles

from desitarget.targetmask import desi_mask, obsconditions

from desitarget.mtl import make_mtl


from fiberassign import __version__ as fba_version

from fiberassign.hardware import (
    load_hardware,
)

from fiberassign.targets import (
    Targets,
    TargetTree,
    TargetsAvailable,
    LocationsAvailable,
    load_target_table,
    default_target_masks,
    TARGET_TYPE_SCIENCE, 
    TARGET_TYPE_SKY,
    TARGET_TYPE_SUPPSKY,
    TARGET_TYPE_STANDARD
)

from fiberassign.tiles import (
    load_tiles,
)

from fiberassign.assign import (
    Assignment,
)

from fiberassign.vis import (
    plot_assignment_tile,
)

from fiberassign.qa import qa_targets

from fiberassign.scripts.assign import (
    parse_assign,
    run_assign_full
)

from fiberassign.scripts.merge import (
    parse_merge,
    run_merge
)


# Capture C++ output in the jupyter cells.
#
# If you want to see the (often very long) output from the underlying calls to fiberassign, then
# uncomment this line (and make sure you have the wurlitzer package installed).
#%reload_ext wurlitzer


#---- Global parameters -----

# Target RA/DEC range
target_ra_min = 158.0
target_ra_max = 202.0
target_dec_min = -2.0
target_dec_max = 32.0

# Date used for the focalplane model
assign_date = "2020-01-01T00:00:00"

# Plotting color
target_pltcolor = {
    "ELG": (0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
    "LRG": (1.0, 0.4980392156862745, 0.054901960784313725),
    "QSO": (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
    "QSO-tracer": (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
    "QSO-lyman": (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
    "STD": (1.0, 0.832, 0.0),
    "SKY": (0.0, 1.0, 1.0)
}

## Data Location

Define the working directories for inputs and outputs, as well as global definitions for the cuts, etc.

In [ ]:
# Working directory (where input and output files will be written).  Change this to wherever you like.
workdir = os.path.join(os.environ["HOME"], "scratch", "desi", "tutorials", "fiberassign_part2")
# workdir = os.path.join(os.environ["SCRATCH"], "desi", "tutorials", "fiberassign_part2")

In [ ]:
# Science / standard target sample
target_sample = os.path.join(workdir, "target_science_sample.fits")

# Sky target sample
target_sample_sky = os.path.join(workdir, "target_sky_sample.fits")

# MTL files for fiberassign
science_file_root = "mtl_science"
science_file = os.path.join(workdir, "{}.fits".format(science_file_root))
std_file = os.path.join(workdir, "mtl_std.fits")
sky_file = os.path.join(workdir, "mtl_sky.fits")

# Footprint
tilefile_root = "footprint"
tilefile_pass = dict()

# Output root directory name for fiberassign
fba_root = "fiberassign"


## Target Selection

Here we grab all targets within an RA / DEC range.  We write this code to a separate python script that can be run outside of this notebook, which is useful if the notebook is running on a laptop or workstation instead of NERSC.  
After running this next cell, copy the generated `select_targets.py` script and run it at NERSC with the default DESI software stack.  Then copy the output FITS files into your working directory (which you defined in the previous cell).

In [ ]:
%%writefile select_targets.py

# Run this script at NERSC with any of the DESI software stacks.  This script only uses fitsio
# and built-in packages.

import os
import glob

import numpy as np

import fitsio

target_ra_min = 158.0
target_ra_max = 202.0
target_dec_min = -2.0
target_dec_max = 32.0

# First select the science targets

# (Change this if you have the DR8 data locally)
input_dir = "/global/cfs/cdirs/desi/target/catalogs/dr8/0.31.1/targets/main/resolve"

input_files = glob.glob(os.path.join(input_dir, "*.fits"))

target_data = []

for file in input_files:
    print("Working on {}".format(os.path.basename(file)), flush=True)
    fd = fitsio.FITS(file, "r")
    fdata = fd[1].read()
    inside = np.where(
        np.logical_and(
            np.logical_and((fdata["RA"] > target_ra_min), (fdata["RA"] < target_ra_max)),
            np.logical_and((fdata["DEC"] > target_dec_min), (fdata["DEC"] < target_dec_max))
        )
    )[0]
    target_data.append(fdata[inside])
    fd.close()

target_data = np.concatenate(target_data)

out_file = "target_science_sample.fits" 
if os.path.isfile(out_file):
    os.remove(out_file)

fd = fitsio.FITS(out_file, "rw")
fd.write(None, header=None, extname="PRIMARY")
fd.write(target_data, header=None, extname="TARGETS")
fd.close()

# Now select the sky targets

print("Working on sky...", flush=True)

sky_file = "/global/cfs/cdirs/desi/target/catalogs/dr8/0.31.0/skies/skies-dr8-0.31.0.fits"

out_file = "target_sky_sample.fits" 
if os.path.isfile(out_file):
    os.remove(out_file)

fd = fitsio.FITS(sky_file, "r")
fdata = fd[1].read()
inside = np.where(
    np.logical_and(
        np.logical_and((fdata["RA"] > target_ra_min), (fdata["RA"] < target_ra_max)),
        np.logical_and((fdata["DEC"] > target_dec_min), (fdata["DEC"] < target_dec_max))
    )
)[0]

outfd = fitsio.FITS(out_file, "rw")
outfd.write(None, header=None, extname="PRIMARY")
outfd.write(fdata[inside], header=None, extname="TARGETS")
outfd.close()

fd.close()


Again, the rest of this notebook uses the target files generated by the previous script (`target_science_sample.fits` and `target_sky_sample.fits`), which it assumes you have placed in the working directory...

In [ ]:
if not os.path.isfile(target_sample) or not os.path.isfile(target_sample_sky):
    message = f'run select_targets.py script and copy outputs to {workdir} before proceeding'
    raise RuntimeError(message)
else:
    print('target sample files generated; ok to proceed')

## Footprint

We take the default DESI footprint and cut out the tiles that fall fully within our RA/DEC limits (up to some buffer) and which have DARK obsconditions.

In [ ]:
footprint_file = dm_findfile("footprint/desi-tiles.fits")
footprint_data = dm_load_tiles(tilesfile=footprint_file, cache=False)

tile_radius = 1.65 # degrees
tile_cut = 2.0 # degrees

tile_ra_min = target_ra_min + tile_cut
tile_ra_max = target_ra_max - tile_cut
tile_dec_min = target_dec_min + tile_cut
tile_dec_max = target_dec_max - tile_cut

obskeep = obsconditions['DARK']

inside = np.where(
    np.logical_and(
        np.logical_and(
            np.logical_and(
                (footprint_data["RA"] > tile_ra_min), 
                (footprint_data["RA"] < tile_ra_max)
            ), np.logical_and(
                (footprint_data["DEC"] > tile_dec_min), 
                (footprint_data["DEC"] < tile_dec_max)
            )
        ),
        (footprint_data["OBSCONDITIONS"] & obskeep)
    )
)[0]

tiledata = footprint_data[inside]

# For each pass, write out a tile file.  Also write out the file for all passes.

passes = np.unique(tiledata["PASS"])

def plot_tiles(tdata, ps, outfile):
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(1, 1, 1)

    ra_width = target_ra_max - target_ra_min
    ra_center = target_ra_min + 0.5 * ra_width
    dec_width = target_dec_max - target_dec_min
    dec_center = target_dec_min + 0.5 * dec_width

    radec_box = plt.Rectangle((target_ra_min, target_dec_min), ra_width, dec_width,
                            angle=0.0, color="green", linewidth=2.0, fill=False)
    ax.add_artist(radec_box)

    for tile in tdata:
        clr = "red"
        if tile["OBSCONDITIONS"] & obsconditions["GRAY"]:
            clr = "cyan"
        outline = plt.Circle(
            (tile["RA"], tile["DEC"]), radius=tile_radius, fc="none", ec=clr
        )
        ax.add_artist(outline)

    ax.set_xlabel("RA", fontsize="large")
    ax.set_ylabel("DEC", fontsize="large")
    ax.set_xlim([target_ra_min - 1.0, target_ra_max + 1.0])
    ax.set_ylim([target_dec_min - 1.0, target_dec_max + 1.0])
    ax.set_title(
        "Tile Selection for pass \"{}\"".format(ps)
    )
    plt.savefig(outfile, dpi=300, format="pdf")
    plt.show()
    

print("Full footprint has {} tiles".format(len(tiledata)))

tilefile_pass["ALL"] = os.path.join(workdir, "{}_ALL.fits".format(tilefile_root))
if os.path.isfile(tilefile_pass["ALL"]):
    os.remove(tilefile_pass["ALL"])

outfd = fitsio.FITS(tilefile_pass["ALL"], "rw")
outfd.write(None, header=None, extname="PRIMARY")
outfd.write(tiledata, header=None, extname="TILES")
outfd.close()

out_pdf = os.path.join(workdir, "{}_ALL.pdf".format(tilefile_root))
plot_tiles(tiledata, "ALL", out_pdf)

for ps in passes:
    pstr = "{}".format(ps)
    ps_rows = np.where(tiledata["PASS"] == ps)[0]
    tiledata_pass = tiledata[ps_rows]
    print("Pass {} footprint has {} tiles".format(ps, len(tiledata_pass)))
    tilefile_pass[pstr] = os.path.join(workdir, "{}_{}.fits".format(tilefile_root, ps))
    if os.path.isfile(tilefile_pass[pstr]):
        os.remove(tilefile_pass[pstr])
    outfd = fitsio.FITS(tilefile_pass[pstr], "rw")
    outfd.write(None, header=None, extname="PRIMARY")
    outfd.write(tiledata_pass, header=None, extname="TILES")
    outfd.close()
    out_pdf = os.path.join(workdir, "{}_{}.pdf".format(tilefile_root, ps))
    plot_tiles(tiledata_pass, ps, out_pdf)

pstr = "2-4"
ps_rows = np.where(tiledata["PASS"] > 1)[0]
tiledata_pass = tiledata[ps_rows]
print("Pass 2-4 footprint has {} tiles".format(len(tiledata_pass)))
tilefile_pass[pstr] = os.path.join(workdir, "{}_2-4.fits".format(tilefile_root))
if os.path.isfile(tilefile_pass[pstr]):
    os.remove(tilefile_pass[pstr])
outfd = fitsio.FITS(tilefile_pass[pstr], "rw")
outfd.write(None, header=None, extname="PRIMARY")
outfd.write(tiledata_pass, header=None, extname="TILES")
outfd.close()
out_pdf = os.path.join(workdir, "{}_{}.pdf".format(tilefile_root, pstr))
plot_tiles(tiledata_pass, pstr, out_pdf)
    

## Working with MTL files

For this exercise we are assigning just the 4 passes of DARK tiles.  We select out only the ELG, LRG and QSO science targets, and also keep targets marked as specific standards (STD_FAINT, STD_WD, STD_BRIGHT).  If there are any targets that have bits for multiple target classes set, we cut those.  In order to avoid confusion, we also remove targets marked as both a standard and a science target.

We will be generating the initial MTL file using the `desitarget.mtl.make_mtl()` function.  We update the MTL file with a small helper function in between runs of fiber assignment.

To make things simpler, we keep only the input target columns needed for assignment.

In [ ]:
# Load the raw science / standard target sample and prune columns

keep_columns = [
    'TARGETID', 
    'RA', 
    'DEC',
    'RA_IVAR',
    'DEC_IVAR',
    'PMRA',
    'PMDEC',
    'PMRA_IVAR',
    'PMDEC_IVAR',
    'DESI_TARGET', 
    'BGS_TARGET', 
    'MWS_TARGET', 
    'SUBPRIORITY', 
    'BRICKNAME',
    'BRICKID',
    'BRICK_OBJID',
    'PRIORITY_INIT', 
    'NUMOBS_INIT'
]

fd = fitsio.FITS(target_sample)
targets_raw = fd[1].read(columns=keep_columns)

# Get the default target masks for this target file

(filesurvey, 
 filecol, 
 def_sciencemask, 
 def_stdmask, 
 def_skymask, 
 def_suppskymask,
 def_safemask, 
 def_excludemask) = default_target_masks(targets_raw)

print("Detected targets for survey '{}', using bitfield column '{}'".format(filesurvey, filecol))

# Force our science and std masks to a more restrictive set.  Only keep ELG, LRG and QSO targets.
# Cut any targets with multiple of those set.

science_mask = 0
science_mask |= desi_mask["LRG"].mask
science_mask |= desi_mask["ELG"].mask
science_mask |= desi_mask["QSO"].mask

std_mask = 0
std_mask |= desi_mask["STD_FAINT"].mask
std_mask |= desi_mask["STD_WD"].mask
std_mask |= desi_mask["STD_BRIGHT"].mask

elg_rows = np.where(
    np.logical_and(
        np.logical_and(
            np.logical_and(
                np.bitwise_and(targets_raw["DESI_TARGET"], desi_mask["ELG"].mask),
                np.logical_not(
                    np.bitwise_and(targets_raw["DESI_TARGET"], std_mask)
                )
            ),
            np.logical_not(
                np.bitwise_and(targets_raw["DESI_TARGET"], desi_mask["QSO"].mask)
            )
        ),
        np.logical_not(
            np.bitwise_and(targets_raw["DESI_TARGET"], desi_mask["LRG"].mask)
        )
    )
)[0]

qso_rows = np.where(
    np.logical_and(
        np.logical_and(
            np.logical_and(
                np.bitwise_and(targets_raw["DESI_TARGET"], desi_mask["QSO"].mask),
                np.logical_not(
                    np.bitwise_and(targets_raw["DESI_TARGET"], std_mask)
                )
            ),
            np.logical_not(
                np.bitwise_and(targets_raw["DESI_TARGET"], desi_mask["ELG"].mask)
            )
        ),
        np.logical_not(
            np.bitwise_and(targets_raw["DESI_TARGET"], desi_mask["LRG"].mask)
        )
    )
)[0]

lrg_rows = np.where(
    np.logical_and(
        np.logical_and(
            np.logical_and(
                np.bitwise_and(targets_raw["DESI_TARGET"], desi_mask["LRG"].mask),
                np.logical_not(
                    np.bitwise_and(targets_raw["DESI_TARGET"], std_mask)
                )
            ),
            np.logical_not(
                np.bitwise_and(targets_raw["DESI_TARGET"], desi_mask["QSO"].mask)
            )
        ),
        np.logical_not(
            np.bitwise_and(targets_raw["DESI_TARGET"], desi_mask["ELG"].mask)
        )
    )
)[0]

n_elg = len(elg_rows)
n_qso = len(qso_rows)
n_lrg = len(lrg_rows)

science_rows = np.concatenate([elg_rows, qso_rows, lrg_rows])

std_rows = np.where(
    np.logical_and(
        np.bitwise_and(targets_raw["DESI_TARGET"], std_mask),
        np.logical_not(
            np.bitwise_and(targets_raw["DESI_TARGET"], science_mask)
        )
    )
)[0]

print(
    "Using {} science and {} standards from input catalog".format(
        len(science_rows),
        len(std_rows)
    )
)

# Split out the science and standard targets, although this is actually not necessary for passing
# to fiberassign.

science_targets = np.array(targets_raw[science_rows])

std_targets = np.array(targets_raw[std_rows])

# Close the input fits file so it doesn't take up extra memory
del targets_raw
fd.close()
del fd

# We have concatenated the 3 target types in the new table, so now the rows are
# different:
elg_rows = np.arange(n_elg, dtype=np.int64)
qso_rows = np.arange(n_qso, dtype=np.int64) + n_elg
lrg_rows = np.arange(n_lrg, dtype=np.int64) + n_elg + n_qso

# Make the MTLs

science_mtl = make_mtl(science_targets, "DARK|GRAY").as_array()
if len(science_mtl) != len(science_targets):
    print("WARNING:  science MTL has {} rows, input has {}".format(len(science_mtl), len(science_targets)))

std_mtl = make_mtl(std_targets, "DARK|GRAY").as_array()
if len(std_mtl) != len(std_targets):
    print("WARNING:  standards MTL has {} rows, input has {}".format(len(std_mtl), len(std_targets)))

# Delete the large intermediate arrays
    
del science_targets
del std_targets
    
# Write MTLs

if os.path.isfile(science_file):
    os.remove(science_file)
with fitsio.FITS(science_file, "rw") as fd:
    fd.write(science_mtl)

if os.path.isfile(std_file):
    os.remove(std_file)
with fitsio.FITS(std_file, "rw") as fd:
    fd.write(std_mtl)    

print("{} science targets".format(len(science_mtl)))
print("    {} ELG targets".format(len(elg_rows)))
print("    {} QSO targets".format(len(qso_rows)))
print("    {} LRG targets".format(len(lrg_rows)))
print("{} std targets".format(len(std_mtl)))

# We'll be loading later science MTLs as we go through the survey, so delete that now.
# the standards are constant so we'll keep those in memory.

del science_mtl


In [ ]:
# Now create the skies file.

keep_columns = [
    'TARGETID', 
    'RA', 
    'DEC', 
    'DESI_TARGET', 
    'BGS_TARGET', 
    'MWS_TARGET', 
    'SUBPRIORITY', 
    'BRICKNAME',
    'BRICKID',
    'BRICK_OBJID',
    'APFLUX_G',
    'APFLUX_R',
    'APFLUX_Z',
    'APFLUX_IVAR_G',
    'APFLUX_IVAR_R',
    'APFLUX_IVAR_Z',
    'OBSCONDITIONS'
]

fd = fitsio.FITS(target_sample_sky)
sky_mtl = np.array(fd[1].read(columns=keep_columns))
fd.close()
del fd

# Sanity check that these are all sky, supp_sky, or bad_sky

print("{} input targets in sky file".format(len(sky_mtl)))

sky_sky_rows = np.where(
    np.bitwise_and(sky_mtl["DESI_TARGET"], desi_mask["SKY"].mask)
)[0]

print("  {} SKY targets".format(len(sky_sky_rows)))

sky_suppsky_rows = np.where(
    np.bitwise_and(sky_mtl["DESI_TARGET"], desi_mask["SUPP_SKY"].mask)
)[0]

print("  {} SUPP_SKY targets".format(len(sky_suppsky_rows)))

sky_badsky_rows = np.where(
    np.bitwise_and(sky_mtl["DESI_TARGET"], desi_mask["BAD_SKY"].mask)
)[0]

print("  {} BAD_SKY targets".format(len(sky_badsky_rows)))

sky_mask = 0
sky_mask |= desi_mask["SKY"].mask
sky_mask |= desi_mask["SUPP_SKY"].mask
sky_mask |= desi_mask["BAD_SKY"].mask

sky_unknown_rows = np.where(
    np.logical_not(
        np.bitwise_and(sky_mtl["DESI_TARGET"], sky_mask)
    )
)[0]

print("  {} targets are not one of the 3 recognized types".format(len(sky_unknown_rows)))

if os.path.isfile(sky_file):
    os.remove(sky_file)
with fitsio.FITS(sky_file, "rw") as fd:
    fd.write(sky_mtl)


## Helper Functions

Now that we have our starting footprint of tiles and our input MTL catalogs, we'll define some functions for use later in this notebook.  These include some convenience wrappers for running the main fiberassign high-level function, for accumulating assignment totals and available / considered targets for a given run, and for updating the MTLs with observation counts.

In [ ]:
# Function to update the MTL obs remaining.

def update_mtl(science_input, science_output, obs):
    """
    This takes the input MTL and sets the NUMOBS_MORE column based on the
    input dictionary of obs remaining for each target.
    """
    
    print("  Loading data from {}".format(science_input), flush=True)
    tdata = None
    with fitsio.FITS(science_input) as fd:
        tdata = fd[1].read()
    
    if "NUMOBS_MORE" not in tdata.dtype.names:
        # create this column based on NUMOBS_INIT
        tdata = append_fields(tdata, "NUMOBS_MORE", tdata["NUMOBS_INIT"])
        
    # Sanity check
    
    if len(obs) != len(tdata):
        msg = "The length of the MTL table ({}) does not match the obs dict ({})".format(
            len(tdata), len(obs)
        )
        raise RuntimeError(msg)
        
    # Now assign the new obs remaining data
    
    print("  Updating observation counts", flush=True)
    tdata["NUMOBS_MORE"][:] = [obs[x] for x in tdata["TARGETID"]]
    
    if os.path.isfile(science_output):
        os.remove(science_output)
        
    print("  Writing updated MTL to {}".format(science_output), flush=True)
    with fitsio.FITS(science_output, "rw") as fd:
        fd.write(tdata)

    del tdata
    return

In [ ]:
# Function to compute the observation count difference between 2 MTLs

def diff_mtl(start_mtl, final_mtl, qso_lyman_rows, qso_tracer_rows):
    keep_columns = [
        "TARGETID",
        "DESI_TARGET",
        "NUMOBS_INIT"
    ]
    print("  Loading data from {}".format(start_mtl), flush=True)
    start_data = None
    with fitsio.FITS(start_mtl) as fd:
        start_data = fd[1].read(columns=keep_columns)
    
    keep_columns = [
        "TARGETID",
        "DESI_TARGET",
        "NUMOBS_MORE"
    ]
    print("  Loading data from {}".format(final_mtl), flush=True)
    final_data = None
    with fitsio.FITS(final_mtl) as fd:
        final_data = fd[1].read(columns=keep_columns)
    
    counts = dict()
    
    for tgclass, rows in zip(
        ["ELG", "LRG", "QSO-lyman", "QSO-tracer"],
        [elg_rows, lrg_rows, qso_lyman_rows, qso_tracer_rows]
    ):
        counts[tgclass] = dict()
        counts[tgclass]["requested"] = np.sum(start_data["NUMOBS_INIT"][rows])
        counts[tgclass]["achieved"] = np.sum(
            start_data["NUMOBS_INIT"][rows] - final_data["NUMOBS_MORE"][rows]
        )
    
    del start_data
    del final_data
    
    return counts

In [ ]:
# Run the fba_run and fba_merge commandline entrypoints

def run_assignment(footprint, outdir, science):
    opts = [
        "--rundate", assign_date,
        "--overwrite",
        "--write_all_targets",
        "--footprint", footprint,
        "--dir", outdir,
        "--targets", science, std_file, sky_file
    ]
    print("  Running raw fiber assignment (fba_run)...")
    print("    (Uncomment the 'wurlitzer' line at the top of the notebook to see the output here)")
    ag = parse_assign(opts)
    run_assign_full(ag)
    
    opts = [
        "--skip_raw",
        "--dir", outdir,
        "--targets", science, std_file, sky_file
    ]
    print("  Merging input target data (fba_merge_results)...")
    print("    (Uncomment the 'wurlitzer' line at the top of the notebook to see the output here)")
    ag = parse_merge(opts)
    run_merge(ag)
    
    return

In [ ]:
# Function to compute the assigned, available, and considered targets for a set of tiles

def assignment_counts(footprint, science_input, fba_dir, qso_lyman_rows, qso_tracer_rows):
    # Load the footprint
    tile_data = None
    with fitsio.FITS(footprint) as fd:
        tile_data = np.array(fd[1].read())
    
    # Load the input science MTL and get the obs remaining for all targets
    mtldata = None
    with fitsio.FITS(science_input) as fd:
        mtldata = fd[1].read()

    obs = None
    if "NUMOBS_MORE" in mtldata.dtype.names:
        obs = {
            x: y for x, y in zip(mtldata["TARGETID"], mtldata["NUMOBS_MORE"])
        }
    else:
        obs = {
            x: y for x, y in zip(mtldata["TARGETID"], mtldata["NUMOBS_INIT"])
        }
    qso_lyman = mtldata["TARGETID"][qso_lyman_rows]
    qso_tracer = mtldata["TARGETID"][qso_tracer_rows]

    class_masks = {
        "ELG": desi_mask["ELG"].mask,
        "LRG": desi_mask["LRG"].mask,
        "QSO-lyman": desi_mask["QSO"].mask,
        "QSO-tracer": desi_mask["QSO"].mask,
        "STD": std_mask,
        "SKY": sky_mask
    }
    
    # histogram data to return
    hist_tgassign = dict()
    hist_tgavail = dict()
    hist_tgconsid = dict()
    hist_tgfrac = dict()
    for tgclass, mask in class_masks.items():
        hist_tgassign[tgclass] = list()
        hist_tgavail[tgclass] = list()
        hist_tgconsid[tgclass] = list()
        hist_tgfrac[tgclass] = list()
    
    print("  Accumulating assignment counts for {} tiles...".format(len(tile_data)), flush=True)
    
    for tl in tile_data["TILEID"]:
        # For each tile in order of assignment...
        
        # Load assignment and available targets and their properties.
        # NOTE: because we used the --write_all_targets option to fba_run, we get the properties
        # of all available targets in the FTARGETS HDU and have access to those here.
        
        fba_file = os.path.join(fba_dir, "fiberassign-{:06d}.fits".format(tl))
        fassign = None
        ftarget = None
        favail = None
        with fitsio.FITS(fba_file, "r") as fd:
            fassign = fd["FIBERASSIGN"].read()
            ftarget = fd["TARGETS"].read()
            favail = fd["POTENTIAL_ASSIGNMENTS"].read()
        
        # The assigned target IDs
        assign_valid_rows = np.where(fassign["TARGETID"] >= 0)[0]
        assign_tgids = np.sort(fassign["TARGETID"][assign_valid_rows])
        assign_target_rows = np.where(
            np.isin(ftarget["TARGETID"], assign_tgids)
        )[0]
        
        # The available target IDs
        avail_tgids = np.sort(np.unique(favail["TARGETID"]))
        avail_target_rows = np.where(
            np.isin(ftarget["TARGETID"], avail_tgids)
        )[0]
        
        # For the science classes, we must also look at the obs remaining
        # in order to know which targets were actually considered for assignment
        # (not just reachable).
        
        for tgclass, mask in class_masks.items():
            # The assigned targets in this class
            assign_class_rows = assign_target_rows[
                np.where(
                    np.bitwise_and(
                        ftarget["DESI_TARGET"][assign_target_rows],
                        mask
                    )
                )[0]
            ]
            if tgclass == "QSO-lyman":
                assign_class_rows = assign_class_rows[
                    np.where(
                        np.isin(
                            ftarget["TARGETID"][assign_class_rows], qso_lyman
                        )
                    )[0]
                ]
            elif tgclass == "QSO-tracer":
                assign_class_rows = assign_class_rows[
                    np.where(
                        np.isin(
                            ftarget["TARGETID"][assign_class_rows], qso_tracer
                        )
                    )[0]
                ]
            
            hist_tgassign[tgclass].append(len(assign_class_rows))
                
            # The available targets in this class
            avail_class_rows = avail_target_rows[
                np.where(
                    np.bitwise_and(
                        ftarget["DESI_TARGET"][avail_target_rows],
                        mask
                    )
                )[0]
            ]
            if tgclass == "QSO-lyman":
                avail_class_rows = avail_class_rows[
                    np.where(
                        np.isin(
                            ftarget["TARGETID"][avail_class_rows], qso_lyman
                        )
                    )[0]
                ]
            elif tgclass == "QSO-tracer":
                avail_class_rows = avail_class_rows[
                    np.where(
                        np.isin(
                            ftarget["TARGETID"][avail_class_rows], qso_tracer
                        )
                    )[0]
                ]

            hist_tgavail[tgclass].append(len(avail_class_rows))
            
            #print("  target class {}, {} assignments".format(tgclass, len(assign_class_rows)))
            
            if tgclass == "STD" or tgclass == "SKY":
                # the considered targets are the same as the reachable
                hist_tgconsid[tgclass].append(len(avail_class_rows))
                hist_tgfrac[tgclass].append(len(assign_class_rows) / len(avail_class_rows))
            else:
                # compare to obs remaining
                hist_tgconsid[tgclass].append(
                    np.sum(
                        [1 for x in ftarget["TARGETID"][avail_class_rows] if obs[x] > 0]
                    )
                )
                hist_tgfrac[tgclass].append(len(assign_class_rows) / hist_tgconsid[tgclass][-1])

                # Now reduce the obs remaining
                for tgid in ftarget["TARGETID"][assign_class_rows]:
                    obs[tgid] -= 1
    
    # Return our histogram of tile data and also the updated observation counts,
    # which can be used to update the MTL NUMOBS_MORE in a separate function.
    return (obs, hist_tgassign, hist_tgavail, hist_tgconsid, hist_tgfrac)


In [ ]:
# This function plots the accumulated assignment counts from a set of tiles.  This is 
# designed to run on the results from one pass / layer.  If you plot the results from
# multiple layers, it will broaden all the distributions, since the considered and assigned
# targets will go down as the survey progresses.

def plot_assignment_stats(
        wd, outname, title,
        hist_tgassign, hist_tgavail, hist_tgconsid, hist_tgfrac
):
    classes = list(sorted(hist_tgavail.keys()))
    fiberbins = 0.01 * np.arange(101)
    tgbins = {
        "ELG": np.arange(0, 22220, 220),
        "LRG": np.arange(0, 4040, 40),
        "QSO-tracer": np.arange(0, 2424, 24),
        "QSO": np.arange(0, 2424, 24),
        "QSO-lyman": np.arange(0, 808, 8),
        "STD": np.arange(0, 1212, 12),
        "SKY": np.arange(0, 40400, 400)
    }

    # The plots
    figfiber = plt.figure(figsize=(12, 6))
    figtarget = plt.figure(figsize=(12, 18))

    # Plot the positioner assignment fractions
    axfiber = figfiber.add_subplot(1, 1, 1)

    for tgclass in classes:
        if np.sum(hist_tgassign[tgclass]) > 0:
            axfiber.hist(
                np.array(hist_tgassign[tgclass]) / 5000, 
                fiberbins, 
                align="mid", 
                alpha=0.8, 
                label="{}".format(tgclass),
                color=target_pltcolor[tgclass]
            )
    axfiber.set_xlabel("Fraction of Assigned Positioners", fontsize="large")
    axfiber.set_ylabel("Tile Counts", fontsize="large")
    axfiber.set_ylim(0, 50)
    axfiber.set_title(
        "Positioner Assignment : {}".format(title)
    )
    axfiber.legend()
    figfiber.savefig(
        os.path.join(wd, "fiberassign_{}_fibers.pdf".format(outname)),
        dpi=300, 
        format="pdf"
    )
    figfiber.show()

    prows = len(classes)
    pcols = 2
    poff = 1
    for tgclass in classes:
        if np.sum(hist_tgavail[tgclass]) == 0:
            continue
        # First the assignment / considered / available plot
        axtarget = figtarget.add_subplot(prows, pcols, poff)
        axtarget.hist(
            np.array(hist_tgavail[tgclass]), 
            tgbins[tgclass], 
            align="mid", 
            alpha=0.4, 
            label="{} Reachable".format(tgclass),
            color=(0.7, 0.7, 0.7)
        )
        axtarget.hist(
            np.array(hist_tgconsid[tgclass]), 
            tgbins[tgclass], 
            align="mid", 
            alpha=0.4, 
            label="{} Considered".format(tgclass),
            color=target_pltcolor[tgclass]
        )
        axtarget.hist(
            np.array(hist_tgassign[tgclass]), 
            tgbins[tgclass], 
            align="mid", 
            alpha=0.8, 
            label="{} Assigned".format(tgclass),
            color=target_pltcolor[tgclass]
        )
        axtarget.set_xlabel("Targets Reachable, Considered, and Assigned", fontsize="large")
        axtarget.set_ylabel("Tile Counts", fontsize="large")
        axtarget.set_ylim(0, 50)
        axtarget.legend()
        poff += 1
        
        # Now the assignment fraction
        axtarget = figtarget.add_subplot(prows, pcols, poff)
        axtarget.hist(
            np.array(hist_tgfrac[tgclass]), 
            fiberbins, 
            align="mid", 
            alpha=0.8, 
            label="Fraction of Considered {} Assigned".format(tgclass),
            color=target_pltcolor[tgclass]
        )
        axtarget.set_xlabel("Fraction of Considered Targets that were Assigned", fontsize="large")
        axtarget.set_ylabel("Tile Counts", fontsize="large")
        axtarget.set_ylim(0, 50)
        axtarget.legend()
        
        poff += 1

    figtarget.suptitle(
        "{}".format(title), fontsize="x-large"
    )
    figtarget.tight_layout(rect=[0, 0, 1, 0.97])
    figtarget.savefig(
        os.path.join(wd, "fiberassign_{}_targets.pdf".format(outname)),
        dpi=300, 
        format="pdf"
    )
    figtarget.show()

In [ ]:
def select_qso_lyman(input_mtl, output_mtl, density, original_mtl, effective_mtl):
    # Set the random seed to ensure reproducibility of this cell
    np.random.seed(123456)

    # Load the input MTL
    print("Loading data from {}".format(input_mtl), flush=True)
    mtldata = None
    with fitsio.FITS(input_mtl) as fd:
        mtldata = fd[1].read()
    
    # The rows with all QSOs was computed globally during the initial MTL creation...
    
    # Find the QSOs we just observed
    qso_observed_rows = np.where(
        np.logical_and(
            np.bitwise_and(
                mtldata["DESI_TARGET"],
                desi_mask["QSO"].mask
            ),
            (mtldata["NUMOBS_MORE"] == 3)
        )
    )[0]
    
    print("  {} total QSO rows".format(len(qso_rows)))
    print("  {} observed QSOs in first pass".format(len(qso_observed_rows)))
    
    # Select a subset of these targets at the correct density.  There are many ways we could
    # do this.  For this exercise we will use an approximate method:
    #
    #    1. Take an inner region of the footprint and compute the target density
    #       of the just-observed QSOs.
    #
    #    2. Take the ratio of the desired density to this value.
    #
    #    3. Use the ratio to get a total number of targets to select from the input.
    #
    #    4. Uniformly sample RA / DEC points and choose nearby objects until we reach
    #       our desired total selection.
    #
    
    # Inner RA/DEC range
    inner_ra_min = target_ra_min + 7.0
    inner_ra_max = target_ra_max - 7.0
    inner_dec_min = target_dec_min + 7.0
    inner_dec_max = target_dec_max - 7.0
    
    inner = np.where(
        np.logical_and(
            np.logical_and(
                (mtldata["RA"][qso_observed_rows] > inner_ra_min), 
                (mtldata["RA"][qso_observed_rows] < inner_ra_max)
            ), np.logical_and(
                (mtldata["DEC"][qso_observed_rows] > inner_dec_min), 
                (mtldata["DEC"][qso_observed_rows] < inner_dec_max)
            )
        )
    )[0]
    
    qso_observed_density = len(inner) / ((inner_ra_max - inner_ra_min) * (inner_dec_max - inner_dec_min))
    
    n_select = int(len(qso_observed_rows) * (density / qso_observed_density))
    
    print("  Selecting {} random QSOs".format(n_select))
    
    # Create a KDTree of the QSOs
    
    kqso = KDTree(
        np.array(list(zip(
            mtldata["RA"][qso_observed_rows], 
            mtldata["DEC"][qso_observed_rows]
        )))
    )
    
    selected = list()
    n_cur = 0
    while n_cur < n_select:
        buf = 10000
        print("  Working on random selection at {} of {} required...".format(n_cur, n_select))
        test_ra = np.random.uniform(low=target_ra_min, high=target_ra_max, size=buf)
        test_dec = np.random.uniform(low=target_dec_min, high=target_dec_max, size=buf)
        dist, indx = kqso.query(np.array(list(zip(test_ra, test_dec))), k=1)
        for i in range(buf):
            if indx[i] in selected:
                continue
            selected.append(indx[i])
            n_cur += 1
            if n_cur == n_select:
                break
    
    selected_rows = np.array(sorted(selected))
    qso_selected_rows = qso_observed_rows[selected_rows]
    
    not_selected_mask = np.ones(len(mtldata), dtype=np.bool)
    not_selected_mask[:] = False
    not_selected_mask[qso_rows] = True
    not_selected_mask[qso_selected_rows] = False
    
    qso_not_selected_rows = np.where(not_selected_mask)[0]
    
    print("  {} QSO-lyman targets".format(len(qso_selected_rows)))
    print("  {} QSO-tracer targets".format(len(qso_not_selected_rows)))
    
    # Plot the selection
    
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(1, 1, 1)
    
    ax.scatter(
        mtldata["RA"][qso_observed_rows], 
        mtldata["DEC"][qso_observed_rows], 
        s=1, c=(0.6, 0.6, 0.6), marker=".",
        label="QSO Observed"
    )
    
    ax.scatter(
        mtldata["RA"][qso_selected_rows], 
        mtldata["DEC"][qso_selected_rows], 
        s=1, c="red", marker=".",
        label="QSO Selected"
    )
    
    ax.set_xlabel("RA", fontsize="large")
    ax.set_ylabel("DEC", fontsize="large")
    ax.set_title("QSOs Observed in Pass 1")
    ax.legend()
    plt.show()
    
    # Set the observations remaining for all other "tracer" QSOs.  For objects
    # that we already observed, these are done.  Other objects will have one observation
    # requested.
    
    mtldata["NUMOBS_MORE"][qso_not_selected_rows] -= 3
    
    # Write new file
    if os.path.isfile(output_mtl):
        os.remove(output_mtl)

    print("Writing updated MTL to {}".format(output_mtl), flush=True)
    with fitsio.FITS(output_mtl, "rw") as fd:
        fd.write(mtldata)
    del mtldata
    
    # In order to get accurate counts based on the future differences between MTLs,
    # we take the original MTL and retro-actively modify NUMOBS_INIT to match our
    # QSO selection
    
    print("Loading original MTL from {}".format(original_mtl), flush=True)
    mtldata = None
    with fitsio.FITS(original_mtl) as fd:
        mtldata = fd[1].read()
    
    mtldata["NUMOBS_INIT"][qso_not_selected_rows] -= 3
    
    print("Writing effective MTL to {}".format(effective_mtl), flush=True)
    if os.path.isfile(effective_mtl):
        os.remove(effective_mtl)
    with fitsio.FITS(effective_mtl, "rw") as fd:
        fd.write(mtldata)
    del mtldata
    
    return (qso_selected_rows, qso_not_selected_rows)

## Running the Survey

In our initial MTL files, all QSOs have 4 requested observations.  We will start by running the first pass / layer.  **Of the QSOs that receive an observation**, we will choose a subset of these (to make up a particular density) for further observations.  All other QSOs will have "3" subtracted from their remaining observations.

In [ ]:
# First pass, using starting MTLs

fiberassign_out = os.path.join(workdir, "fiberassign_pass_1") 
run_assignment(tilefile_pass["1"], fiberassign_out, science_file)

In [ ]:
# Accumulate counts

(obs, hist_assign, hist_avail, hist_consid, hist_frac) = assignment_counts(
    tilefile_pass["1"], science_file, fiberassign_out, list(), qso_rows
)

In [ ]:
# Plot

plot_assignment_stats(
    workdir, "pass_1", "Assignment for Pass 1",
    hist_assign, hist_avail, hist_consid, hist_frac
)

In [ ]:
# Update MTL

mtl_after1 = os.path.join(workdir, "{}_after_pass_1.fits".format(science_file_root))
update_mtl(science_file, mtl_after1, obs)

In [ ]:
# Count the number of observations of each target class so far.  We have not selected QSOs for
# Further observations yet, so all of the QSOs are "tracers" at the moment.

obs_counts = diff_mtl(science_file, mtl_after1, list(), qso_rows)
for tgclass, props in obs_counts.items():
    print("{:10s} : {:7d} observations of {:7d} requested ({:0.1f}%)".format(
        tgclass, props["achieved"], props["requested"], 100*(props["achieved"] / props["requested"])
    ))


Now that we have completed the first pass, we want to select which of the just-observed QSOs we will choose for further observations.  The QSOs which now have "3" remaining observations are the ones we just observed.

In [ ]:
mtl_after_qso = os.path.join(workdir, "{}_after_QSO_select.fits".format(science_file_root))
effective_science_file = os.path.join(workdir, "{}_effective.fits".format(science_file_root))

qso_lyman_rows, qso_tracer_rows = select_qso_lyman(
    mtl_after1, 
    mtl_after_qso, 
    60.0,
    science_file,
    effective_science_file
)


### Passes 2 - 4

Now we can loop over our passes / layers and assign each one

In [ ]:
mtl_output = mtl_after_qso

for survey_pass in ["2", "3", "4"]:
    print("Working on Survey Pass {}".format(survey_pass), flush=True)
    mtl_input = mtl_output
    
    fiberassign_out = os.path.join(workdir, "fiberassign_pass_{}".format(survey_pass)) 
    run_assignment(tilefile_pass[survey_pass], fiberassign_out, mtl_input)
    
    (obs, hist_assign, hist_avail, hist_consid, hist_frac) = assignment_counts(
        tilefile_pass[survey_pass], 
        mtl_input, 
        fiberassign_out,
        qso_lyman_rows, 
        qso_tracer_rows
    )
    
    plot_assignment_stats(
        workdir, 
        "pass_{}".format(survey_pass), 
        "Assignment for Pass {}".format(survey_pass),
        hist_assign, hist_avail, hist_consid, hist_frac
    )
    
    mtl_output = os.path.join(workdir, "{}_after_pass_{}.fits".format(science_file_root, survey_pass))
    update_mtl(mtl_input, mtl_output, obs)
    
    obs_counts = diff_mtl(effective_science_file, mtl_output, qso_lyman_rows, qso_tracer_rows)
    for tgclass, props in obs_counts.items():
        print("{:10s} : {:7d} observations of {:7d} requested ({:0.1f}%)".format(
            tgclass, props["achieved"], props["requested"], 100*(props["achieved"] / props["requested"])
        ))
    